In [5]:
import pandas as pd
import glob
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf

In [6]:
from pmdarima.arima import auto_arima

In [7]:
import pmdarima as pm

In [8]:
from prophet import Prophet

In [9]:
data = pd.read_excel('consumo_energia_industrial_mg.xlsx')
data.shape

(117, 5)

In [10]:
data.head()

,Time,UF,Descrição,Setor,Consumo_MWh
0,2013-1-1,Minas Gerais,Consumo de energia elétrica na rede (MWh),Industrial,2429892.0
1,2013-2-1,Minas Gerais,Consumo de energia elétrica na rede (MWh),Industrial,2392524.0
2,2013-3-1,Minas Gerais,Consumo de energia elétrica na rede (MWh),Industrial,2398782.0
3,2013-4-1,Minas Gerais,Consumo de energia elétrica na rede (MWh),Industrial,2542895.0
4,2013-5-1,Minas Gerais,Consumo de energia elétrica na rede (MWh),Industrial,2523754.0


In [11]:
data['Time'] = pd.to_datetime(data['Time'].str.strip(), format='%Y-%m-%d')
data['Consumo_MWh'] = data['Consumo_MWh'] / 1000

In [12]:
data = data[['Time', 'Consumo_MWh']]
data.head()

,Time,Consumo_MWh
0,2013-01-01,2429.892
1,2013-02-01,2392.524
2,2013-03-01,2398.782
3,2013-04-01,2542.895
4,2013-05-01,2523.754


In [13]:
data24 = data[(data['Time'] > '2020-09-01')]
data12 = data[(data['Time'] > '2021-09-01')]
data6 = data[(data['Time'] > '2022-03-01')]

In [14]:
data24 = data24.rename(columns={"Time": "ds", "Consumo_MWh": "y"})

In [15]:
data24.head()

,ds,y
93,2020-10-01,2728.95042
94,2020-11-01,2611.13640
95,2020-12-01,2625.10557
96,2021-01-01,2725.74494
97,2021-02-01,2553.74788


In [16]:
data24.shape

(24, 2)

In [17]:
data24.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [18]:
TEST_SIZE = 6
train, test = data24.iloc[:-TEST_SIZE], data24.iloc[-TEST_SIZE:]
x_train, x_test = np.array(range(train.shape[0])), np.array(range(train.shape[0], data24.shape[0]))
train.shape, x_train.shape, test.shape, x_test.shape

((18, 2), (18,), (6, 2), (6,))

In [19]:
train.head(), train.tail(), train.shape

(           ds           y
 93 2020-10-01  2728.95042
 94 2020-11-01  2611.13640
 95 2020-12-01  2625.10557
 96 2021-01-01  2725.74494
 97 2021-02-01  2553.74788,
             ds           y
 106 2021-11-01  2758.66041
 107 2021-12-01  2762.81263
 108 2022-01-01  2675.56891
 109 2022-02-01  2478.88357
 110 2022-03-01  2512.58318,
 (18, 2))

In [20]:
test

,ds,y
111,2022-04-01,2692.27648
112,2022-05-01,2826.89462
113,2022-06-01,2560.89226
114,2022-07-01,2629.56443
115,2022-08-01,2734.19473
116,2022-09-01,2528.52568


In [21]:
m = Prophet()
m.fit(train)

15:37:21 - cmdstanpy - INFO - Chain [1] start processing
15:37:21 - cmdstanpy - INFO - Chain [1] done processing


In [22]:
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
 date = '2022-%02d' % i
 future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds'] = pd.to_datetime(future['ds'])

future = future[(future['ds'] > '2022-03-01')]
future = future[(future['ds'] < '2022-10-01')]
future = future.reset_index()
future = future[['ds']]

In [23]:
future

,ds
0,2022-04-01
1,2022-05-01
2,2022-06-01
3,2022-07-01
4,2022-08-01
5,2022-09-01


In [24]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2022-04-01,2581.791536,2479.018465,2685.200807
1,2022-05-01,2558.948784,2456.675674,2656.366188
2,2022-06-01,2535.344607,2430.608397,2633.215991
3,2022-07-01,2512.501855,2423.792092,2614.458587
4,2022-08-01,2488.897678,2387.370671,2586.908318
5,2022-09-01,2465.293501,2360.386104,2575.261374


In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)